In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
!pip install transformers
!pip install SentencePiece
import torch
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from transformers import (AutoConfig, 
                          AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, 
                          get_linear_schedule_with_warmup,
                          set_seed,
                          )
from keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')


     |████████████████████████████████| 2.5MB 8.5MB/s 
     |████████████████████████████████| 3.3MB 20.5MB/s 
     |████████████████████████████████| 901kB 48.3MB/s 
     |████████████████████████████████| 1.2MB 8.1MB/s 
Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/colab_data'
train = pd.read_csv(os.path.join(path, "fake_job_postings.csv"))
train['jd'] = train[['title','function','employment_type','company_profile','description','requirements','benefits']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_train, df_val = train_test_split(train, test_size=0.33, random_state=42)
print(np.quantile(df_train.jd.apply(lambda x: len(x)),0.95))
df_train.head(3)

5365.500000000002


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,jd
13014,13015,Digital Senior Account Manager,"GB, LND, London",Client Services,NaN,"Working in over 60 countries and 30 languages,...",Role SummaryAutotorq is looking for a self-mot...,Key Skills/Experience:Digital marketing experi...,NaN,0,1,0,Full-time,Director,NaN,Automotive,Marketing,0,Digital Senior Account Manager Marketing Full-...
12094,12095,Sales Rep for AT&T Solutions Provider - Manage...,"US, TX, Arlington",AFSDT,45000-67000,"Argenta Field Solutions values the client, cre...",Interviewing now for full-time positions in Ar...,- Sales experience preferred- Ability to work ...,- AFLAC- Health Insurance (Management) - Train...,0,1,0,Full-time,NaN,NaN,Consumer Services,Sales,0,Sales Rep for AT&T Solutions Provider - Manage...
16090,16091,Investor Development Analyst,NaN,NaN,NaN,MarketInvoice is one of the most high-profile ...,MarketInvoice is one of the most high-profile ...,Experience equivalent to a minimum of 2 years ...,Full time role based in our London Holborn off...,0,1,1,Full-time,Associate,NaN,Financial Services,Business Development,0,Investor Development Analyst Business Developm...


In [8]:
# model_name = 'bert-base-cased'
# model_name = 'xlnet-base-cased'
# model_name = 'xlm-roberta-large'
model_name = 'bert-base-uncased'
n_labels = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




In [9]:
class Transformer_Dataset(Dataset):

    def __init__(self, text, target, tokenizer, max_len):
        self.text = text
        self.target = target
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item):
        text = str(self.text[item])
        target = self.target[item]

        encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'text': text,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'target': torch.tensor(target, dtype=torch.long)
        }

def create_data_loader(df, text, target, tokenizer, max_len, batch_size):
  ds = Transformer_Dataset(
    text=df[text].to_numpy(),
    target=df[target].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2,
    drop_last = True
  )

# Get model configuration.
print('Loading configuraiton...')
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name, 
                                          num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Get the actual model.
print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name, 
                                                           config=model_config)

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)



Loading configuraiton...
Loading tokenizer...
Loading model...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model loaded to `cuda`


In [15]:
set_seed(123)
epochs = 4
batches = 10
max_len = 512
class_names = ['no_fraud','fraud']
output_model = 'research_cat.bin'
train_data_loader = create_data_loader(df_train,'jd', 'fraudulent', tokenizer, max_len, batches)
val_data_loader = create_data_loader(df_val, 'jd', 'fraudulent', tokenizer, max_len, batches)

In [16]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [17]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples, batch_size, maxlen):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(batch_size,maxlen).to(device)
        attention_mask = d["attention_mask"].to(device)
        target = d['target'].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = target)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        target = target.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(target, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

def eval_model(model, data_loader, device, n_examples,batch_size,maxlen):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(batch_size,maxlen).to(device)
            attention_mask = d["attention_mask"].to(device)
            target = d['target'].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = target)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            target = target.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(target, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [18]:

%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in tqdm(range(epochs)):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train),
        batches,
        max_len
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val),
        batches,
        max_len
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), os.path.join(path, output_model))
        best_accuracy = val_acc

Epoch 1/4
----------
Train loss 0.1801425774823703 Train accuracy 0.9572263993316592
Val loss 0.11365080986456093 Val accuracy 0.9725423728813537

Epoch 2/4
----------
Train loss 0.10136969387035276 Train accuracy 0.9773600668337494
Val loss 0.07805963823556016 Val accuracy 0.9813559322033882

Epoch 3/4
----------
Train loss 0.05327418409719746 Train accuracy 0.9895572263993311
Val loss 0.06732603605901744 Val accuracy 0.9859322033898292

Epoch 4/4
----------
Train loss 0.026411552594499735 Train accuracy 0.9954887218045112
Val loss 0.06847450527879488 Val accuracy 0.9876271186440665


CPU times: user 1h 39min 28s, sys: 1min 5s, total: 1h 40min 34s
Wall time: 1h 40min 57s


In [19]:
def predict_sentiment(text):
    text = text

    encoded_review = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=max_len,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
    )

    input_ids = pad_sequences(encoded_review['input_ids'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_review['attention_mask'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,max_len).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)
    # return class_names[prediction]
    return prediction
    # print("Positive score:", probs[1])
    # print("Negative score:", probs[0])
    # print("Neutral score", probs[2])
    # print(f'text: {text}')
    # print(f'target  : {class_names[prediction]}')

In [20]:
df_val['pred']=df_val.jd.apply(lambda x: int(predict_sentiment(x)))
print(confusion_matrix(df_val.fraudulent, df_val.pred))
print(classification_report(df_val.fraudulent, df_val.pred))

[[5581   21]
 [  52  247]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5602
           1       0.92      0.83      0.87       299

    accuracy                           0.99      5901
   macro avg       0.96      0.91      0.93      5901
weighted avg       0.99      0.99      0.99      5901



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
